# *Gaia*-2MASS cross-match

## Depreciated

APOGEE Contains 2MASS photometry, no need to cross-match with 2MASS.

In [1]:
import astropy.units as u
from astropy.table import Table
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [ ]:
Gaia.login()

In [3]:
path = '../data/serenelli2017_gaia_4arcsec.csv'

df = pd.read_csv(path)
print(len(df))
df.head()

453


,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,...,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist,nearest_neighbour
0,1635721458409799680,Gaia DR2 2051714265379610624,2051714265379610624,1642703359,2015.5,292.082639,0.022439,37.059874,0.026381,7.287584,...,DSEP,0,0,0,7.983,7.753,7.718,none,0.076482,1
1,1635721458409799680,Gaia DR2 2051083248784888576,2051083248784888576,951146959,2015.5,290.505591,0.020091,37.458935,0.026062,1.578592,...,DSEP,0,1,1,10.427,10.204,10.155,cand,0.068342,1
2,1635721458409799680,Gaia DR2 2051847684243759104,2051847684243759104,789196893,2015.5,292.901449,0.019937,37.624510,0.027366,2.547900,...,DSEP,0,0,1,10.040,9.604,9.500,none,0.032582,1
3,1635721458409799680,Gaia DR2 2051667914096024448,2051667914096024448,1066183801,2015.5,293.073631,0.021554,37.716582,0.026427,3.636056,...,DSEP,0,0,0,9.585,9.333,9.283,none,0.031511,1
4,1635721458409799680,Gaia DR2 2099526837747055232,2099526837747055232,1298060034,2015.5,287.548574,0.023019,38.082134,0.027463,8.247791,...,DSEP,0,0,0,7.649,7.481,7.464,none,0.122110,1


In [6]:
# cols = ['kepid', 'source_id', 'tm_designation']
# table = Table().from_pandas(df[cols])

In [7]:
# job = Gaia.launch_job(
#     "SELECT * \
#     FROM gaiadr2.gaia_source as g, gaiadr2.tmass_best_neighbour as tm\
#     INNER JOIN tap_upload.table AS t \
#         ON g.source_id = t.gaia AND tm.source_id = t.gaia",
#     upload_resource=table,
#     upload_table_name='table'
# )

In [8]:
# results = job.get_results()
# gaia = results.to_pandas()
# print(len(gaia))
# gaia.head()

In [9]:
# gaia['tmass_source_id'] = '2MASS J' + gaia['original_ext_source_id'].str.decode("utf-8")
# gaia['tmass_source_id'].head()

In [10]:
gaia = df

In [11]:
jobs = []
for obj_id in gaia['tm_designation'].to_numpy():
    jobs.append(
        Vizier.query_object(object_name=obj_id, catalog='II/246/out', radius=4.0*u.arcsec)
    )

In [12]:
jobs[:5]

[TableList with 1 tables:
 	'0:II/246/out' with 15 column(s) and 1 row(s) , TableList with 1 tables:
 	'0:II/246/out' with 15 column(s) and 1 row(s) , TableList with 1 tables:
 	'0:II/246/out' with 15 column(s) and 1 row(s) , TableList with 1 tables:
 	'0:II/246/out' with 15 column(s) and 1 row(s) , TableList with 1 tables:
 	'0:II/246/out' with 15 column(s) and 1 row(s) ]

In [21]:
tmass_dfs = []
for job in jobs:
    tmass_dfs.append(job['II/246/out'].to_pandas())

In [22]:
tmass = pd.concat(tmass_dfs)
tmass.head()

,RAJ2000,DEJ2000,_2MASS,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Qflg,Rflg,Bflg,Cflg,Xflg,Aflg
0,292.082690,37.059811,b'19281984+3703353',7.983,0.024,7.753,0.023,7.718,0.017,b'AAA',b'111',b'111',b'000',0,0
0,290.505646,37.459015,b'19220135+3727324',10.427,0.020,10.204,0.021,10.155,0.019,b'AAA',b'222',b'111',b'000',0,0
0,292.901507,37.624481,b'19313636+3737281',10.040,0.022,9.604,0.020,9.500,0.018,b'AAA',b'222',b'111',b'000',0,0
0,293.073563,37.716461,b'19321765+3742592',9.585,0.020,9.333,0.019,9.283,0.018,b'AAA',b'222',b'111',b'000',0,0
0,287.548468,38.082199,b'19101163+3804559',7.649,0.021,7.481,0.016,7.464,0.027,b'AAA',b'111',b'111',b'000',0,0


In [24]:
# gaia_tmass = gaia.merge(tmass, left_on='original_ext_source_id', right_on='_2MASS')
# gaia_tmass.head()

In [25]:
# total = df.merge(gaia_tmass, left_on='KIC', right_on='kic', suffixes=['', ''])
# total.head()

In [26]:
# total.to_csv('../data/serenelli2017-berger2018-gaia-2mass_all.csv', index=False)

In [29]:
tmass = tmass.reset_index(drop=True)

In [40]:
tmass['tm_designation'] = '2MASS J' + tmass['_2MASS'].str.decode("utf-8")

In [41]:
tmass['tm_designation']

0      2MASS J19281984+3703353
1      2MASS J19220135+3727324
2      2MASS J19313636+3737281
3      2MASS J19321765+3742592
4      2MASS J19101163+3804559
                ...           
451    2MASS J19420696+5031392
452    2MASS J19410030+5041414
453    2MASS J19401696+5054134
454    2MASS J19463773+5101135
455    2MASS J19442066+5107373
Name: tm_designation, Length: 456, dtype: object

In [ ]:
gaia = gaia.rename(columns={'source_id': 'gaia_source_id'})

In [60]:
tmass_gaia = gaia[['KIC', 'tm_designation', 'gaia_source_id', 'nearest_neighbour']].merge(tmass, on='tm_designation')
tmass_gaia.head()

,RAJ2000,DEJ2000,_2MASS,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Qflg,Rflg,Bflg,Cflg,Xflg,Aflg,tm_designation,source_id,kepid,nearest_neighbour
0,292.082690,37.059811,b'19281984+3703353',7.983,0.024,7.753,0.023,7.718,0.017,b'AAA',b'111',b'111',b'000',0,0,2MASS J19281984+3703353,2051714265379610624,1435467,1
1,290.505646,37.459015,b'19220135+3727324',10.427,0.020,10.204,0.021,10.155,0.019,b'AAA',b'222',b'111',b'000',0,0,2MASS J19220135+3727324,2051083248784888576,2010607,1
2,292.901507,37.624481,b'19313636+3737281',10.040,0.022,9.604,0.020,9.500,0.018,b'AAA',b'222',b'111',b'000',0,0,2MASS J19313636+3737281,2051847684243759104,2309595,1
3,293.073563,37.716461,b'19321765+3742592',9.585,0.020,9.333,0.019,9.283,0.018,b'AAA',b'222',b'111',b'000',0,0,2MASS J19321765+3742592,2051667914096024448,2450729,1
4,287.548468,38.082199,b'19101163+3804559',7.649,0.021,7.481,0.016,7.464,0.027,b'AAA',b'111',b'111',b'000',0,0,2MASS J19101163+3804559,2099526837747055232,2837475,1


In [63]:
tmass_gaia.to_csv('../data/archive/s17_gaia_4arcsec_2mass.csv', index=False)